# Solution Report

This file aims to study each approach, detailing it's main strategy as well as weighting in on it's advantages and disadvantages

**Important Note:** If you want to run this report, be sure you are using a **Linux** distribution

### Index
 - [Setting up the Environment](#setting-up-the-environment)
 - [Brute-Force](#brute-force)

## Setting up the Environment

To generate the example graphs, as well as checking if the cliques we found are correct, the `networkx` library is used. Let's make sure that this library is installed

In [8]:
%pip install networkx

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Note that there aren't any graphs in the directory currently. Let's generate them using the `generate_graph.py` in the `scripts` folder. We will generate 6 sets of graphs with different sizes and densities:
 - `small_low_density.txt`
 - `medium_low_density.txt`
 - `large_low_density`

In [9]:
# generate_graph.py args: argv[1] = n_edges, argv[2] = probability of connection, argv[3] = filename
%run scripts/generate_graph.py 20 0.3 test

Graph generated and saved at /home/gabrielhso/Desktop/insper/2023.2/superComp/maximum-clique-detection/graphs'.


## Brute-Force

### Description
The brute force approach to finding the maximum clique in a graph is a method that systematically explores all possible combinations of vertices to identify the largest complete subgraph, known as a clique. It starts with an initial vertex and iteratively adds vertices to a candidate clique, checking if the addition maintains the clique property. This exhaustive approach guarantees the optimality of the solution, but its computational complexity grows exponentially with the number of vertices, making it impractical for large graphs.

### Running the Code

As we will evaluate the efficiency of the algorithim itself, we won't compile it with the optimization options of the compiler (-O flag). Also, we will use the -g to include debug symbols.


In [23]:
# Compiling
!g++ -g scripts/brute_force.cpp -o ./scripts/brute_force

In [24]:
# Running
!./scripts/brute_force graphs/test.txt

Size of the maximum clique: 4
Maximum clique found:
[1, 3, 5, 18]


Let's check the answer using the `check_answer.py` file

In [25]:
%run scripts/check_answer.py graphs/test.txt

Size of the maximum clique:  4
Maximum cliques found:
[1, 3, 5, 18]
[2, 7, 10, 12]


### Profiling

Now let's check the performance of this code

In [26]:
!g++ -Wall -O0 -g scripts/brute_force.cpp -o scripts/brute_force

In [27]:
!valgrind --tool=callgrind ./scripts/brute_force

==52406== Callgrind, a call-graph generating cache profiler
==52406== Copyright (C) 2002-2017, and GNU GPL'd, by Josef Weidendorfer et al.
==52406== Using Valgrind-3.18.1 and LibVEX; rerun with -h for copyright info
==52406== Command: ./scripts/brute_force
==52406== 
==52406== For interactive control, run 'callgrind_control -h'.
Run the program with the graph file as an argument (e.g., ./main graph.txt)
==52406== 
==52406== Events    : Ir
==52406== Collected : 2345238
==52406== 
==52406== I   refs:      2,345,238


In [4]:
!callgrind_annotate callgrind.out.23473 scripts/brute_force.cpp

--------------------------------------------------------------------------------
Profile data file 'callgrind.out.23473' (creator: callgrind-3.18.1)
--------------------------------------------------------------------------------
I1 cache: 
D1 cache: 
LL cache: 
Timerange: Basic block 0 - 376659
Trigger: Program termination
Profiled target:  ./scripts/brute_force (PID 23473, part 1)
Events recorded:  Ir
Events shown:     Ir
Event sort order: Ir
Thresholds:       99
Include dirs:     
User annotated:   scripts/brute_force.cpp
Auto-annotation:  on

--------------------------------------------------------------------------------
Ir                 
--------------------------------------------------------------------------------
2,345,648 (100.0%)  PROGRAM TOTALS

--------------------------------------------------------------------------------
Ir                file:function
--------------------------------------------------------------------------------
987,882 (42.12%)  ./elf/./elf/dl-lo